# GPT Action Library: SQLデータベース

## はじめに

これは、GPT Actionを使用してChatGPTにSQLデータベースをクエリする機能を提供したい開発者向けのガイドです。このガイドを読む前に、以下のコンテンツに慣れ親しんでください：

* [GPT Actionsの紹介](https://platform.openai.com/docs/actions)
* [GPT Actions Libraryの紹介](https://platform.openai.com/docs/actions/actions-library)
* [GPT Actionをゼロから構築する例](https://platform.openai.com/docs/actions/getting-started)

このガイドでは、ミドルウェアアプリケーションを介してChatGPTをSQLデータベースに接続するために必要なワークフローを説明します。この例ではPostgreSQLデータベースを使用しますが、プロセスはすべてのSQLデータベース（MySQL、MS SQL Server、Amazon Aurora、Google Cloud上のSQL Serverなど）で同様のはずです。このドキュメントでは、以下が可能なGPT Actionを作成するために必要な手順を説明します：

* SQLデータベースに対する読み取りクエリの実行
* テキストレスポンスによるレコードの返却
* CSVファイルによるレコードの返却

### 価値 + ビジネスユースケースの例

**価値**: ユーザーはChatGPTの自然言語機能を活用して、SQLデータベース内のデータに関する質問に答えることができるようになります：

* ビジネスユーザーは、SQLを書いたりアナリストにリクエストを送信したりすることなく、SQLデータベースに含まれる情報にアクセスできます
* データアナリストは、データを抽出してChatGPTで分析することで、SQLクエリだけでは不可能な複雑な分析を実行できます

**ユースケースの例**:

* ビジネスユーザーが営業ファネルについて質問に答える必要がある
* データアナリストが大規模なデータセットで回帰分析を実行する必要がある

## アプリケーション設計の考慮事項

ほとんどのマネージドSQLデータベースはクエリを送信するためのREST APIを提供していないため、以下の機能を実行するミドルウェアアプリケーションが必要になります：

1. REST APIリクエストを介してデータベースクエリを受け取る
2. 統合されたSQLデータベースにクエリを転送する
3. データベースレスポンスをCSVファイルに変換する
4. CSVファイルをリクエスト元に返却する

最初の機能を設計するには、主に2つのアプローチがあります：

1. ミドルウェアがGPTによって生成された任意のSQLクエリを受信し、それらをデータベースに転送する単一のメソッドをサポートする。このアプローチの利点は以下の通りです：
    1. 開発の容易さ
    2. 柔軟性（ユーザーが行うクエリのタイプを予測する必要がない）
    3. 低メンテナンス（データベースの変更に応じてAPIスキーマを更新する必要がない）

2. ミドルウェアが特定の許可されたクエリに対応する複数のメソッドをサポートする。このアプローチの利点は以下の通りです：
    4. より多くの制御
    5. SQLを生成する際のモデルエラーの機会が少ない

このガイドではオプション1に焦点を当てます。オプション2に興味がある方は、プロセスを合理化するために[PostgREST](https://github.com/PostgREST/postgrest)や[Hasura](https://hasura.io/)のようなサービスの実装を検討してください。

![ユーザー、GPT、ミドルウェア、データベース間の相互作用を示すアプリケーションアーキテクチャ図](../../../images/gptactions_sql_database_middleware.png)
_アプリケーションアーキテクチャ図_

## ミドルウェアの考慮事項

開発者は、カスタムミドルウェア（一般的にAWS、GCP、MS AzureなどのCSPでサーバーレス関数としてデプロイされる）を構築するか、サードパーティソリューション（[Mulesoft Anypoint](https://www.mulesoft.com/platform/enterprise-integration)や[Retool Workflows](https://retool.com/products/workflows)など）を使用することができます。サードパーティミドルウェアを使用すると開発プロセスを加速できますが、自分で構築するよりも柔軟性は低くなります。

独自のミドルウェアを構築すると、アプリケーションの動作をより制御できます。カスタムミドルウェアの例については、[Azure Functions cookbook](https://cookbook.openai.com/examples/chatgpt/gpt_actions_library/gpt_middleware_azure_function)をご覧ください。

ミドルウェアセットアップの詳細に焦点を当てるのではなく、このガイドではミドルウェアのGPTおよびSQLデータベースとのインターフェースに焦点を当てます。

## ワークフローステップ

### 1) GPTがSQLクエリを生成する

GPTは、ユーザーの自然言語プロンプトに基づいてSQLクエリを書くのが非常に得意です。以下のいずれかの方法でデータベーススキーマへのアクセスを提供することで、GPTのクエリ生成機能を向上させることができます：

1. GPTにスキーマを取得するためにまずデータベースをクエリするよう指示する（このアプローチは[BigQuery cookbook](https://cookbook.openai.com/examples/chatgpt/gpt_actions_library/gpt_action_bigquery#custom-gpt-instructions)でより詳しく説明されています）。
2. GPTの指示にスキーマを提供する（小さな静的スキーマに最適）

以下は、シンプルなデータベーススキーマに関する情報を含むGPT指示のサンプルです：

In [ ]:
# Context
You are a data analyst. Your job is to assist users with their business questions by analyzing the data contained in a PostgreSQL database.

## Database Schema

### Accounts Table
**Description:** Stores information about business accounts.

| Column Name  | Data Type      | Constraints                        | Description                             |
|--------------|----------------|------------------------------------|-----------------------------------------|
| account_id   | INT            | PRIMARY KEY, AUTO_INCREMENT, NOT NULL | Unique identifier for each account      |
| account_name | VARCHAR(255)   | NOT NULL                           | Name of the business account            |
| industry     | VARCHAR(255)   |                                    | Industry to which the business belongs  |
| created_at   | TIMESTAMP      | NOT NULL, DEFAULT CURRENT_TIMESTAMP | Timestamp when the account was created  |

### Users Table
**Description:** Stores information about users associated with the accounts.

| Column Name  | Data Type      | Constraints                        | Description                             |
|--------------|----------------|------------------------------------|-----------------------------------------|
| user_id      | INT            | PRIMARY KEY, AUTO_INCREMENT, NOT NULL | Unique identifier for each user         |
| account_id   | INT            | NOT NULL, FOREIGN KEY (References Accounts(account_id)) | Foreign key referencing Accounts(account_id) |
| username     | VARCHAR(50)    | NOT NULL, UNIQUE                   | Username chosen by the user             |
| email        | VARCHAR(100)   | NOT NULL, UNIQUE                   | User's email address                    |
| role         | VARCHAR(50)    |                                    | Role of the user within the account     |
| created_at   | TIMESTAMP      | NOT NULL, DEFAULT CURRENT_TIMESTAMP | Timestamp when the user was created     |

### Revenue Table
**Description:** Stores revenue data related to the accounts.

| Column Name  | Data Type      | Constraints                        | Description                             |
|--------------|----------------|------------------------------------|-----------------------------------------|
| revenue_id   | INT            | PRIMARY KEY, AUTO_INCREMENT, NOT NULL | Unique identifier for each revenue record |
| account_id   | INT            | NOT NULL, FOREIGN KEY (References Accounts(account_id)) | Foreign key referencing Accounts(account_id) |
| amount       | DECIMAL(10, 2) | NOT NULL                           | Revenue amount                          |
| revenue_date | DATE           | NOT NULL                           | Date when the revenue was recorded      |

# Instructions:
1. When the user asks a question, consider what data you would need to answer the question and confirm that the data should be available by consulting the database schema.
2. Write a PostgreSQL-compatible query and submit it using the `databaseQuery` API method.
3. Use the response data to answer the user's question.
4. If necessary, use code interpreter to perform additional analysis on the data until you are able to answer the user's question.

### 2) GPTがミドルウェアにSQLクエリを送信

GPTがミドルウェアと通信するために、GPT Actionを設定します。ミドルウェアは、SQLクエリ文字列を受け取るREST APIエンドポイントを提供する必要があります。このインターフェースはいくつかの方法で設計できます。以下は、POST操作で"q"パラメータを受け取るシンプルなエンドポイントのOpenAPIスキーマの例です：

In [ ]:
openapi: 3.1.0
info:
  title: PostgreSQL API
  description: API for querying a PostgreSQL database
  version: 1.0.0
servers:
  - url: https://my.middleware.com/v1
    description: middleware service
paths:
  /api/query:
    post:
      operationId: databaseQuery
      summary: Query a PostgreSQL database
      requestBody:
        required: true
        content:
          application/json:
            schema:
              type: object
              properties:
                q:
                  type: string
                  example: select * from users
      responses:
        "200":
          description: database records
          content:
            application/json:
              schema:
                type: object
                properties:
                  openaiFileResponse:
                    type: array
                    items:
                      type: object
                      properties:
                        name:
                          type: string
                          description: The name of the file.
                        mime_type:
                          type: string
                          description: The MIME type of the file.
                        content:
                          type: string
                          format: byte
                          description: The content of the file in base64 encoding.
        "400":
          description: Bad Request. Invalid input.
        "401":
          description: Unauthorized. Invalid or missing API key.
      security:
        - ApiKey: []
components:
  securitySchemes:
    ApiKey:
      type: apiKey
      in: header
      name: X-Api-Key
  schemas: {}

**認証に関する注意事項：** 上記の例のAPIインターフェースは、GPTの設定と共に保存され、すべてのGPTユーザーのリクエストを認証するために使用される単一のシステムレベルAPIキーを受け入れます。GPT Actionsは、ユーザーレベルの認証と認可を可能にするOAuth認証もサポートしています。[GPT Actionの認証オプションについて詳しく学ぶ](https://platform.openai.com/docs/actions/authentication)。

ユーザーは基盤となるデータベースと直接ではなく、ミドルウェアで認証を行うため、ユーザーレベルのアクセス（テーブルまたは行レベルの権限）を強制するには、より多くの労力が必要です。ただし、ユーザーが基盤となるデータベースに対して異なるレベルのアクセス権を持つGPTでは、これが必要になる場合があります。

ユーザーレベルの権限を強制するために、ミドルウェアは以下を行う必要があります：

1. OAuthフロー中にIdPから提供されるユーザーのメタデータを受信し、識別情報を抽出する
2. データベースにクエリを実行してユーザーのデータベース権限を取得する
3. セッションの残りの期間、関連する権限を強制するためのコマンドをデータベースに発行する

良好なユーザーエクスペリエンスを維持するために、スキーマデータをGPTの指示に直接含めるのではなく、各ユーザーに対して利用可能なデータベーススキーマを動的に取得することをお勧めします。これにより、GPTが現在のユーザーの代わりにクエリできるテーブルのみにアクセスできることが保証されます。

### 3) ミドルウェアがSQLクエリをデータベースに転送

ミドルウェアは、PostgreSQLデータベースに直接クエリを実行できるようにするため、データベースドライバーまたはクライアントライブラリを実装します。サードパーティのミドルウェアを使用している場合、ミドルウェアベンダーはSQLデータベース用のネイティブコネクタを提供する必要があります。独自のミドルウェアを構築している場合は、データベースベンダーまたはサードパーティが提供するクライアントライブラリを実装する必要がある場合があります。例えば、PostgreSQL用のコミュニティメンテナンスのクライアントライブラリのリストは以下にあります：[https://wiki.postgresql.org/wiki/List_of_drivers](https://wiki.postgresql.org/wiki/List_of_drivers)

このワークフローステップでは、ミドルウェアアプリケーションは、GPTから受信したリクエストからSQL文字列を抽出し、クライアントライブラリが提供するメソッドを使用してデータベースに転送する必要があります。

**読み取り専用権限に関する注意事項：** この設計パターンでは、データベースが任意のAI生成SQLクエリを処理することになるため、ミドルウェアアプリケーションがデータベースに対して読み取り専用権限を持つことを確認する必要があります。これにより、AI生成クエリが新しいデータを挿入したり、既存のデータを変更したりできないことが保証されます。ユースケースで書き込みアクセスが必要な場合は、任意のSQLを受け入れるのではなく、操作固有のエンドポイントをデプロイすることを検討してください。

### 4) データベースがレコードをミドルウェアに返す

実装したクライアントライブラリによって、ミドルウェアは様々な形式でレコードを受信する場合があります。一般的なパターンの一つは、ミドルウェアがJSONオブジェクトの配列を受信することで、各オブジェクトがクエリに一致するデータベースレコードを表します：

In [ ]:
[
  {
    "account_id": 1,
    "number_of_users": 10,
    "total_revenue": 43803.96,
    "revenue_per_user": 4380.40
  },
  {
    "account_id": 2,
    "number_of_users": 12,
    "total_revenue": 77814.84,
    "revenue_per_user": 6484.57
  },
  ...
]

### 5) ミドルウェアがレコードをbase64エンコードされたCSVファイルに変換する

ChatGPTが大量のレコードを分析するためには、CSV形式のデータにアクセスする必要があります。GPT ActionsインターフェースはGPTが最大10mbまでの[base64エンコードされたファイルを受信](https://platform.openai.com/docs/actions/sending-files/returning-files)することを可能にします。

あなたのミドルウェアは2つのアクションを実行する必要があります：


#### レコードをCSV形式に変換する

多くのプログラミング言語には、CSVファイルを扱うためのネイティブライブラリが含まれています（例えば、Pythonの[csv](https://docs.python.org/3/library/csv.html)ライブラリなど）。

以下は、ミドルウェアがJSONオブジェクトの配列をCSVファイルに変換する方法の例です：

In [ ]:
import json
import csv

# Sample JSON array of objects
json_data = '''
[
    {"account_id": 1, "number_of_users": 10, "total_revenue": 43803.96, "revenue_per_user": 4380.40}, 
    {"account_id": 2, "number_of_users": 12, "total_revenue": 77814.84, "revenue_per_user": 6484.57}
]
'''

# Load JSON data
data = json.loads(json_data)

# Define the CSV file name
csv_file = 'output.csv'

# Write JSON data to CSV
with open(csv_file, 'w', newline='') as csvfile:
    # Create a CSV writer object
    csvwriter = csv.writer(csvfile)
    
    # Write the header (keys of the first dictionary)
    header = data[0].keys()
    csvwriter.writerow(header)
    
    # Write the data rows
    for row in data:
        csvwriter.writerow(row.values())

print(f"JSON data has been written to {csv_file}")

#### CSVファイルをBase64エンコードする

多くのプログラミング言語には、base64エンコーディングを扱うためのネイティブライブラリが含まれています（例えば、Pythonの[base64](https://docs.python.org/3/library/base64.html)ライブラリなど）。

以下は、前のステップで生成されたCSVファイルをミドルウェアでbase64エンコードする方法の例です：

In [ ]:
import base64 

# Base64 encode the CSV file
encoded_string = base64.b64encode(open('output.csv', 'rb').read()).decode('utf-8')

print("Base64 Encoded CSV:")
print(encoded_string)

### 6) ミドルウェアがbase64エンコードされたCSVファイルをGPTに返す

GPT Actionsインターフェースがbase64エンコードされたCSVファイルを処理するために、ミドルウェアから返されるレスポンスには`openaiFileResponse`パラメータが含まれている必要があります。提供される値は、ファイルオブジェクトの配列またはファイルへのリンクでなければなりません（詳細については[Actionsドキュメント](https://platform.openai.com/docs/actions/sending-files/returning-files)を参照してください）。この例では、ファイルオブジェクトの配列を使用します。

有効なレスポンスボディの例は以下のようになります：

In [ ]:
{
  "openaiFileResponse": [
    {
      "name": "output.csv",
      "mime_type": "text/csv",
      "content": "ImFjY291bn...NC41NyI="
    }
  ]
}

### 7) GPTが返されたファイルを処理する

GPTがbase64エンコードされたCSVファイルを受信すると、自動的にファイルをデコードし、ユーザーの質問に答えるためにファイルを処理します。これには、CSVファイルに対して[code interpreterを使用した追加の分析を実行する](https://help.openai.com/en/articles/9213685-extracting-insights-with-chatgpt-data-analysis)ことが含まれる場合があり、これはユーザーがプロンプト経由でCSVファイルをアップロードした場合と同じように動作します。

**注意：** 返されたファイルに対して追加の分析を実行できるようにしたい場合は、GPTで_Code Interpreter & Data Analysis_機能を有効にする必要があります。


## 結論

GPT Actionsは、SQLデータベースなどの外部ソースからデータを取得するための柔軟なフレームワークを提供します。ChatGPTにデータベースをクエリする能力を与えることで、ナレッジアシスタントおよびアナリストとしての機能を大幅に拡張できます。

_優先してほしい統合はありますか？統合にエラーはありますか？githubでPRやissueを提出していただければ、確認いたします。_